# Tableau Data Source Q&A

Now that you learned how to create an agent using the prebuilt `create_react_agent` in the [first exercise](./01_intro_to_langgraph.ipynb), let's provide it with a Tableau tool so it can query a datasource and analyze the result. 

This Datasource Q&A tool leverages a few Tableau features to function:

1. [VizQL Data Service API](https://help.tableau.com/current/api/vizql-data-service/en-us/index.html) -> Where the Agent requests data from a published data source and obtains useful metadata
2. [Metadata API](https://help.tableau.com/current/api/metadata_api/en-us/index.html) -> How the Agent obtains useful semantics such as synonyms for individual fields via the Data Catalog

With these capabilities you can equip the agent to surface insights in natural language from the data sources you have diligently built to run your analytics practice with [Tableau Prep](https://www.tableau.com/products/prep) (pictured below).

![Tableau Prep](./assets/prep_builder.png)

This is the perfect way to maximize the impact of your work with AI agents.

For this exercise we will create a **Superstore Agent** that is able to query the [Superstore Datasource](https://help.tableau.com/current/guides/get-started-tutorial/en-us/get-started-tutorial-connect.htm). This is convenient because this data source is available on every [Tableau Cloud](https://www.tableau.com/products/cloud-bi) or [Tableau Server](https://www.tableau.com/products/server) site by default and it also ships with every [Tableau Desktop](https://www.tableau.com/products/desktop) app.

If you need a personal Tableau site for development purposes refer to [this guide](https://medium.com/@cristiansaavedra/how-to-get-your-tableau-online-sandbox-61d6a1f8b697).

Start by importing the necessary packages:

In [2]:
# to access environment variables
import os
from dotenv import load_dotenv

# for displaying pretty results in the notebook
from IPython.display import display, Markdown

# Langgraph packages
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent

# langchain_tableau packages
from langchain_tableau.tools.simple_datasource_qa import initialize_simple_datasource_qa

## Environment Variables

In order to query published data sources on Tableau, the agent tool must authenticate to the site using a [Connected App](https://www.tableau.com/blog/unlock-power-personalized-analytics-user-attribute-functions). This authentication mechanism is the best practice for security and control when integrating code with Tableau as it provides the most control to developers.

Connected Apps [rely on secrets and IDs](https://help.tableau.com/current/online/en-us/connected_apps_direct.htm) to "sign" [JSON Web Tokens](https://jwt.io/) (JWTs) that should be kept private and not published to the internet on places such as Github. Other bad actors such as people and agents can find these credentials and use them for nefarios purposes. 

In order to secure these credentials we use the industry standard `.env` file which is ignored by `git` and therefore not published to places like Github when we share code.

The following code has the `load_dotenv()` function load the contents of the `.env` file so it is accessible via Python code and then used them to initialize the variables we need for the tool:

In [3]:
# loads environment variables into Python script
load_dotenv()

# initialize variables with these secure values
tableau_server = os.getenv('TABLEAU_DOMAIN')
tableau_site = os.getenv('TABLEAU_SITE')
tableau_jwt_client_id = os.getenv('TABLEAU_JWT_CLIENT_ID')
tableau_jwt_secret_id = os.getenv('TABLEAU_JWT_SECRET_ID')
tableau_jwt_secret = os.getenv('TABLEAU_JWT_SECRET')
tableau_api_version = os.getenv('TABLEAU_API_VERSION')
tableau_user = os.getenv('TABLEAU_USER')
datasource_luid = os.getenv('DATASOURCE_LUID')
tooling_model = os.getenv('TOOLING_MODEL')

## Initialize the Tool

We first start a tool with the `initialize_simple_datasource_qa` function providing it with the information it needs to query a Tableau data source securely.

Notice how the tool requires knowledge of a server, a site, the Connected App credentials (JWT), a user, a data source luid and a model. This is necessary in order for it to access a site as a specified user, find the right data source and then use an LLM to translate natural language questions or tasks the Agent receives into a data query.

Once the tool is ready, we then add this tool to a "List" or Python data structure in brackets `[]` to contain one or more tools for the agent to use:

In [4]:
# Initalize the tool for querying Tableau Datasources through VDS
analyze_datasource = initialize_simple_datasource_qa(
    domain=tableau_server,
    site=tableau_site,
    jwt_client_id=tableau_jwt_client_id,
    jwt_secret_id=tableau_jwt_secret_id,
    jwt_secret=tableau_jwt_secret,
    tableau_api_version=tableau_api_version,
    tableau_user=tableau_user,
    datasource_luid=datasource_luid,
    tooling_llm_model=tooling_model
)

# add the tool to a List to give to the agent
tools = [ analyze_datasource ]

## The Superstore Agent

It is finally time to build our Superstore Agent with access to the [Superstore Data Source](https://help.tableau.com/current/guides/get-started-tutorial/en-us/get-started-tutorial-connect.htm).

After initializing the tool we can proceed start an agent using the prebuilt `create_react_agent` function provided by Langchain. This function has three basic requirements: a model, a list of tools and a prompt.

The LLM model in this case is used as the "brains" of the agent. This model chooses what tool to use according to the task is has been assigned to execute. In this capacity the LLM is relied upon to reason, plan and execute actions to accomplish a job:

In [7]:
# initialize a Large Language Model to be the "brains" of the Agent
model = ChatOpenAI(model='gpt-4o-mini', temperature=0)

The prompt will give the agent clear instructions on its role, how to behave, what tools to prioritize and even give it a personality!

In [8]:
prompt = """
Identity:
You are a Superstore Agent able to perform tasks that require access to Superstore's sales and shipping data.
Therefore you are also an Analyst designed to generate data-driven insights to provide answers, guidance and analysis
to humans and other AI Agents.

Tool Choice:
1. Query Data Source: performs ad-hoc queries and analysis. Prioritize this tool for most requests, especially if
the user explicitly asks for data queries or fetches. This tool is great for getting values for specific dates, for
breakdowns by category, for aggregations such as AVG and MAX, for filtered results, etc.
specific data values such as values on a specific date

Restrictions:
- DO NOT HALLUCINATE data if they are not provided via available tools

Output:
Your output should be structured conversationally being short and concise unless you have been asked for a report
Always answer the question first and then provide any additional details or insights
"""


We the initialize the Superstore Agent with these three basic components:

In [12]:
superstore_agent = create_react_agent(model=model, tools=tools, prompt=prompt)

## Run the Agent

When the Superstore Agent is asked to do something and it decides to query a data source, it will provide inputs to the tool to describe what data it needs to analyze and the tool translates these requirements into a query to obtain data from Superstore to answer your questions

In [13]:
# Run the agent
messages = superstore_agent.invoke({
    "messages": [("human",'which states sell the most? Are those the same states with the most profits?')]
})

# display a nicely formatted answer
display(Markdown(messages['messages'][4].content)) 

The states with the highest sales and profits are as follows:

### Top States by Sales:
1. **California**: $457,687.63
2. **New York**: $310,876.27
3. **Texas**: $170,188.05
4. **Washington**: $138,641.27
5. **Pennsylvania**: $116,511.91

### Top States by Profit:
1. **California**: $76,381.39
2. **New York**: $74,038.55
3. **Washington**: $33,402.65
4. **Michigan**: $24,463.19
5. **Virginia**: $18,597.95

### Comparison:
- **California** and **New York** are the top states in both sales and profits.
- **Texas**, while having high sales, ranks lower in profits, indicating potential issues with cost management or pricing strategies.

If you need further analysis or insights, feel free to ask!